In [1]:
from sqlalchemy import create_engine, exc
import pandas as pd
from datetime import datetime, timedelta

In [2]:
usuario = 'root'
senha = 'root'
host = 'localhost'
port = 3308
banco = 'vendas_dimensional'

In [3]:
datainicial = datetime(1900, 1, 1)
datafinal = datetime(2200, 12, 31)

# Listas auxiliares
meses = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
         'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')
dias = ('Domingo', 'Segunda-feira', 'Terça-feira', 'Quarta-feira',
        'Quinta-feira', 'Sexta-feira', 'Sábado')

In [4]:
try:
    engine = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{port}/{banco}")
    with engine.connect() as connection:
        print(f"Conexão com o banco '{banco}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{banco}': {e}")
    exit()

Conexão com o banco 'vendas_dimensional' estabelecida com sucesso.


In [5]:
data = datainicial
id = 1
dim_tempo = pd.DataFrame()
while data <= datafinal:
    linha = {
        'id': id,
        'data': data.date(),
        'dia': data.day,
        'mes': data.month,
        'ano': data.year,
        'data_juliana': str(data.year) + data.strftime('%j'),
        'semestre': 1 if data.month <= 6 else 2,
        'quadrimestre': (data.month - 1) // 4 + 1,
        'trimestre': (data.month - 1) // 3 + 1,
        'bimestre': (data.month - 1) // 2 + 1,
        'nome_mes': meses[data.month - 1],
        'dia_da_semana': (data.weekday() + 1) % 7 + 1,  # 1=Domingo
        'nome_dia_da_semana': dias[(data.weekday() + 1) % 7],
        'semana_do_ano': data.isocalendar()[1],
        'data_string': data.strftime('%Y-%m-%d'),
        'dia_no_ano': data.timetuple().tm_yday,
        'ultimo_dia_mes': (data + pd.offsets.MonthEnd(0)).day,
        'fim_de_semana': int(data.weekday() >= 5)
    }
    temp_df = pd.DataFrame([linha])
    dim_tempo = pd.concat([dim_tempo,temp_df])
    id += 1
    data += timedelta(days=1)
try:
    dim_tempo.to_sql('dim_tempo', con=engine, if_exists='append', index=False)
except exc.SQLAlchemyError as e:
    print(f"Erro ao inserir a data {data.strftime('%Y-%m-%d')}: {e}")

print("Dimensão tempo carregada com sucesso.")


Dimensão tempo carregada com sucesso.
